# SAM3 for FiftyOne - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/sam3_images/blob/main/sam3_quickstart.ipynb)

Complete guide to using Meta's SAM3 (Segment Anything Model 3) in FiftyOne.

**Features:**
- 🎯 Concept Segmentation (text prompts)
- 🖱️ Visual Segmentation (interactive prompts)
- 🤖 Automatic Segmentation (no prompts)
- 🔍 Visual Embeddings (1024-dim)


## 1. Installation

SAM3 requires transformers from source (not yet on PyPI):


In [ ]:
%pip install -q git+https://github.com/huggingface/transformers.git#egg=transformers
%pip install -q fiftyone torch torchvision huggingface-hub umap-learn


## 2. Setup


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

# Load dataset
dataset = foz.load_zoo_dataset("quickstart", max_samples=200)

# Register SAM3 zoo model
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/sam3_images"
)

# Load model
model = foz.load_zoo_model("facebook/sam3")

print(f"✅ Loaded {len(dataset)} samples")


## 3. Visual Embeddings

Extract 1024-dim embeddings for similarity search:


In [ ]:
# Compute embeddings
model.pooling_strategy = "max"  # or "mean", "cls"

dataset.compute_embeddings(
    model,
    embeddings_field="sam_embeddings",
    batch_size=32
)

print("✅ Embeddings computed")


In [ ]:
# Visualize with UMAP
fob.compute_visualization(
    dataset,
    method="umap",
    brain_key="sam_viz",
    embeddings="sam_embeddings",
    num_dims=2
)

print("✅ UMAP visualization ready")


## 4. Concept Segmentation

Find ALL instances matching text concepts:


In [ ]:
# Single concept
model.operation = "concept_segmentation"
model.prompt = "person"
model.threshold = 0.5
model.mask_threshold = 0.5

dataset.apply_model(
    model,
    label_field="people",
    batch_size=16,
    num_workers=4
)


In [ ]:
# Multiple concepts (runs once per concept - slower!)
# 7 concepts = 7x inference passes
model.prompt = [
    "bird", "human", "land vehicle", "air vehicle",
    "animal", "food", "furniture"
]

dataset.apply_model(
    model,
    label_field="multi_concept",
    batch_size=8,
    num_workers=4
)


## 5. Visual Segmentation

Refine existing detections into precise masks:


In [ ]:
model.operation = "visual_segmentation"

dataset.apply_model(
    model,
    label_field="visual_masks",
    prompt_field="ground_truth",  # Use existing detections as prompts
    batch_size=64,
    num_workers=4
)


## 6. Automatic Segmentation

Segment everything with quality filtering:


In [ ]:
model.operation = "automatic_segmentation"
model.auto_kwargs = {
    "points_per_side": 16,       # Grid: 16x16 = 256 points
    "points_per_batch": 256,     # Batch size
    "quality_threshold": 0.8,    # Keep high quality only
    "iou_threshold": 0.85,       # Remove duplicates
    "max_masks": 100             # Top 100 masks
}

# Run on subset (automatic is slow)
dataset.take(10).apply_model(
    model,
    label_field="auto_masks",
    batch_size=2,
    num_workers=2
)



## 7. Launch FiftyOne App


In [ ]:
session = fo.launch_app(dataset)

# In Colab, use:
# session = fo.launch_app(dataset, auto=True)
# print(session.url)


## 8. Explore Results


In [ ]:
from fiftyone import ViewField as F

# Filter by label
people = dataset.filter_labels("people", F("label") == "person")
print(f"Found people in {len(people)} samples")

# Filter by confidence
high_conf = dataset.filter_labels("multi_concept", F("confidence") > 0.8)
print(f"High confidence detections: {len(high_conf.exists('multi_concept'))}")

# Similarity search
fob.compute_similarity(dataset, embeddings="sam_embeddings")
query = dataset.first()
similar = dataset.sort_by_similarity(query, k=10)
print(f"Found {len(similar)} similar images")


## Summary

You've now:
- ✅ Computed visual embeddings for similarity search
- ✅ Found objects using text prompts (concept segmentation)
- ✅ Refined detections with precise masks (visual segmentation)
- ✅ Generated all possible masks (automatic segmentation)

**Next Steps:**
- Try different prompts for concept segmentation
- Adjust thresholds for more/fewer detections
- Experiment with automatic segmentation parameters

**Documentation:** https://github.com/harpreetsahota204/sam3_images
